In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math as mt
import os
import pickle
import mlflow
import mlflow.keras
import keras_tuner as kt

In [3]:
pip install keras-tuner --upgrade

Note: you may need to restart the kernel to use updated packages.


In [23]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Physical devices cannot be modified after being initialized


In [3]:


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("graph-neural-hyperparameter-trial-1")

<Experiment: artifact_location='./mlruns/5', experiment_id='5', lifecycle_stage='active', name='graph-neural-hyperparameter-trial-1', tags={}>

In [4]:
mlflow.tensorflow.autolog()

In [5]:
with open('data/train/x_train.pkl','rb') as f:
        x1=pickle.load(f)
with open('data/train/y_train.pkl','rb') as f:
        y1=pickle.load( f)
with open('data/valid/x_valid.pkl','rb') as f:
        x2=pickle.load(f)
with open('data/valid/y_valid.pkl','rb') as f:
        y2=pickle.load( f)
with open('data/test/x_test.pkl','rb') as f:
        x3=pickle.load(f)
with open('data/test/y_test.pkl','rb') as f:
        y3=pickle.load( f)

In [14]:
class GCN(tf.keras.layers.Layer):
    def __init__(self,inp_shape=9*12,out_shape=9*12,h=9, w=12,near=3):
        super(GCN, self).__init__()
        self.am=[[0 for i in range(w*h)] for j in range(h*w)]
        self.img=[[0 for i in range(w)] for j in range(h)]
  
        self.dr=[[1,0],[0,1],[1,1],[-1,-1],[-1,0],[0,-1],[-1,1],[1,-1]]

        for i,v in enumerate(self.img):
           for j,v2 in enumerate(self.img[i]):
        # timg=[[0 for i in range(w)] for j in range(h)]
              for k in range(0,near+1):
                for d in self.dr:
                  posy=i+d[0]*k
                  posx=j+d[1]*k
                  if posy>=0 and posy<h and posx>=0 and posx<w:
            # timg[posy][posx]=1
                    self.am[i*w+j][posy*w+posx]=1
        # print(timg) 
        self.am=np.array(self.am) 
        self.D = np.diag(np.sum(self.am, axis=0)) 
        self.Dinv=np.linalg.inv(self.D)
        self.am=tf.Variable(self.am,trainable=True,dtype=tf.float32,name="ajm")
        self.Dinv=tf.Variable(self.Dinv,trainable=True,dtype=tf.float32,name="dm")
        # self.norm=tf.matmul(self.Dinv*self.am)
        self.w = self.add_weight(
            shape=(inp_shape, out_shape), initializer="random_normal", trainable=True,dtype=tf.float32,name='adjw'
        )
        # self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)
    
    def call(self, inputs):
        inputs=tf.cast(inputs,dtype=tf.float32)
        # inputs=tf.transpose()
        print(inputs.shape)
        # print("inpshape",tf.transpose(inputs,perm=[1,0]).shape)
        # print(self.Dinv.shape)
        # print(self.am.shape)
        # print(self.w.shape)
        first_half=tf.matmul(inputs,tf.matmul(self.Dinv,self.am))
    
        return tf.matmul(first_half, self.w) 

In [18]:
def build_model(hp,near=3,lay=2,bs=2,n_out=7):
    inputs = tf.keras.Input( shape=(9,12),)
    # print(inputs.shape)
    near=hp.Choice("number of nearest neighbour", [2, 3,4])
    # mlflow.log_param("nearest", near)
    x=tf.keras.layers.Flatten()(inputs)
    grl=hp.Int( "no of layers for graph",min_value=lay, max_value=lay+15, step=4)
    for i in range(grl):
        # mlflow.log_param("no of layers for graph", val)
        x=GCN(near=near)(x)
        x=tf.keras.layers.Activation('relu',name="graph_layer"+str(i+1))(x)
        
    x=tf.keras.layers.BatchNormalization()(x)
    val=hp.Int( "no of layers for graph",min_value=lay, max_value=lay+15, step=4)
    for i in range(val):
        ut=hp.Int('units', min_value=32, max_value=128, step=32)
        al=hp.Choice("activation", ["relu", "tanh"])
        x=tf.keras.layers.Dense(units=ut,activation=al,name="dense"+str(i))(x)
        with mlflow.start_run(nested=True):
            mlflow.log_param("no of layers for dense", val)
            mlflow.log_param("no of units", ut)
            mlflow.log_param("type of activation in dense", al)
    output=tf.keras.layers.Dense(n_out,activation="softmax",name="out_layer")(x)
    model = tf.keras.Model(inputs=inputs, outputs=output, name="graph_neuralnetwork")
    lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    metrics=["accuracy"],
     )
    with mlflow.start_run(nested=True):
         mlflow.log_param("nearest", near)
         mlflow.log_param("no of layers for dense", val)
         mlflow.log_param("no of layers for graph", grl)
         mlflow.log_param("learning_rate", val)
    return model

In [19]:

hp = kt.HyperParameters()

In [20]:
with mlflow.start_run(nested=True):
    model=build_model(hp)

(None, 108)
(None, 108)


In [21]:
with mlflow.start_run(nested=True):
    tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="model/hyperparam",
    project_name="graph_neural_net",
)

(None, 108)
(None, 108)


In [ ]:
tuner.search_space_summary()

In [22]:

tuner.search(x1, y1, epochs=2, validation_data=(x2, y2))

Trial 3 Complete [00h 00m 08s]
val_accuracy: 0.1428571492433548

Best val_accuracy So Far: 0.14642857760190964
Total elapsed time: 00h 00m 34s
INFO:tensorflow:Oracle triggered exit


In [24]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]

(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)


In [25]:
best_model.summary()

Model: "graph_neuralnetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 12)]           0         
                                                                 
 flatten (Flatten)           (None, 108)               0         
                                                                 
 gcn (GCN)                   (None, 108)               34992     
                                                                 
 graph_layer1 (Activation)   (None, 108)               0         
                                                                 
 gcn_1 (GCN)                 (None, 108)               34992     
                                                                 
 graph_layer2 (Activation)   (None, 108)               0         
                                                                 
 gcn_2 (GCN)                 (None, 108)       

In [ ]:
best_model.save('/Users/ariqrahman/Desktop/Mac-Neural-Network-Project/model/hyperparam/gcn-hyp')